In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/code") 

In [9]:
model_checkpoint = 'uer/t5-base-chinese-cluecorpussmall'
model_self = 'sunhao666/chi-sum2'

In [2]:
! pip install transformers datasets rouge-score

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


### 读取文本

In [10]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='./nlpcc_data.json', field='data')

Using custom data configuration default-2603363f9a670f5e
Reusing dataset json (/Users/sunhao/.cache/huggingface/datasets/json/default-2603363f9a670f5e/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)


In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 50000
    })
})

In [12]:
train_test = dataset['train'].train_test_split(test_size=0.2)
train_vali = train_test['test'].train_test_split(test_size=0.5)
train_test['vali'] = train_vali['train']
train_test['test'] = train_vali['test']
dataset = train_test
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 5000
    })
    vali: Dataset({
        features: ['title', 'content'],
        num_rows: 5000
    })
})

In [13]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(dataset["train"])

TypeError: 'int' object is not callable

### 开始处理

In [21]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
len(dataset['train'][0]['content'])

TypeError: 'int' object is not callable

In [23]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["content"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [24]:
preprocess_function(dataset['train'][:2])

{'input_ids': [[101, 2802, 782, 1957, 2094, 2864, 678, 4638, 2813, 2813, 6158, 2802, 1400, 4638, 4212, 4275, 127, 3299, 124, 3189, 117, 2813, 2336, 2356, 4638, 2408, 6205, 5093, 1957, 2111, 2813, 2813, 113, 1265, 1399, 114, 4917, 117, 126, 3299, 8110, 3189, 3241, 677, 117, 1762, 2813, 2336, 2356, 671, 7368, 6205, 1277, 1278, 7368, 8115, 3517, 4567, 2791, 1079, 117, 671, 855, 679, 6371, 6399, 4638, 1957, 2094, 2190, 1961, 1157, 2797, 3318, 679, 719, 4638, 5567, 6956, 671, 7561, 3274, 2802, 117, 5307, 7063, 2137, 711, 6768, 839, 511, 100, 2218, 1728, 711, 2769, 3146, 2159, 3146, 2533, 1962, 4692, 117, 2218, 6421, 2917, 2802, 1408, 136, 100, 2813, 2813, 4917, 117, 1961, 2347, 5470, 6435, 2526, 2360, 5335, 3326, 511, 6381, 5442, 3219, 3189, 113, 127, 3299, 124, 3189, 114, 749, 6237, 1168, 117, 4680, 1184, 2813, 2336, 100, 3736, 6356, 3175, 2347, 792, 1057, 6444, 3389, 511, 2496, 752, 5442, 6432, 1157, 3146, 2130, 2159, 4638, 5567, 6901, 7359, 4495, 782, 3274, 2802, 100, 2496, 3198, 2769, 2

In [25]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

### Fine tuning模型

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_self)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
# batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
    fp16=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

In [27]:

import lawrouge
rouge = lawrouge.Rouge()
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    score_result = rouge.get_scores([predictions], [labels], avg=0)[0] 
    return score_result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["vali"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
# trainer.train()

RuntimeError: ignored

In [ ]:
tokenized_datasets["test"].num_rows

5000

In [28]:
rouge1 = []
rouge2 = []
rougeL = []
len = dataset['test'].num_rows
for i, data in enumerate(dataset['test']):
  sentence = data['content']
  summary = data['title']
  input = tokenizer(sentence, max_length=max_target_length, truncation=True, return_tensors='pt')
  del input['token_type_ids']
  output = model.generate(
    **input,
    do_sample=True,
    num_beams=3,
    bad_words_ids=[[101], [100]],
    # length_penalty=100,
    max_length=100,
    repetition_penalty=5.0
  )[0]
  sum_sys = tokenizer.decode(output).split('[SEP]')[0].replace('[CLS]', '').replace(' ', '')
  try:
    score = compute_metrics((sum_sys, summary))
  except:
    continue
  rouge1.append(score['rouge-1'])
  rouge2.append(score['rouge-2'])
  rougeL.append(score['rouge-l'])
  print('='*10, ':', f'{i} / {len}')

========== : 0 / 5000
========== : 1 / 5000
========== : 2 / 5000
========== : 3 / 5000
========== : 4 / 5000
========== : 5 / 5000
========== : 6 / 5000
========== : 7 / 5000
========== : 8 / 5000
========== : 9 / 5000
========== : 10 / 5000
========== : 11 / 5000
========== : 12 / 5000
========== : 13 / 5000
========== : 14 / 5000
========== : 15 / 5000
========== : 16 / 5000
========== : 17 / 5000
========== : 18 / 5000
========== : 19 / 5000
========== : 20 / 5000
========== : 21 / 5000
========== : 22 / 5000
========== : 23 / 5000
========== : 24 / 5000
========== : 25 / 5000
========== : 26 / 5000
========== : 27 / 5000
========== : 28 / 5000
========== : 29 / 5000
========== : 30 / 5000
========== : 31 / 5000
========== : 32 / 5000
========== : 33 / 5000
========== : 34 / 5000
========== : 35 / 5000
========== : 36 / 5000
========== : 37 / 5000
========== : 38 / 5000
========== : 39 / 5000
========== : 40 / 5000
========== : 41 / 5000
========== : 42 / 5000
========== : 43 / 500

========== : 348 / 5000
========== : 349 / 5000
========== : 350 / 5000
========== : 351 / 5000
========== : 352 / 5000
========== : 353 / 5000
========== : 354 / 5000
========== : 355 / 5000
========== : 356 / 5000
========== : 357 / 5000
========== : 358 / 5000
========== : 359 / 5000
========== : 360 / 5000
========== : 361 / 5000
========== : 362 / 5000
========== : 363 / 5000
========== : 364 / 5000
========== : 365 / 5000
========== : 366 / 5000
========== : 367 / 5000
========== : 368 / 5000
========== : 369 / 5000
========== : 370 / 5000
========== : 371 / 5000
========== : 372 / 5000
========== : 373 / 5000
========== : 374 / 5000
========== : 375 / 5000
========== : 376 / 5000
========== : 377 / 5000
========== : 378 / 5000
========== : 379 / 5000
========== : 380 / 5000
========== : 381 / 5000
========== : 382 / 5000
========== : 383 / 5000
========== : 384 / 5000
========== : 385 / 5000
========== : 386 / 5000
========== : 387 / 5000
========== : 388 / 5000
========== : 389

========== : 692 / 5000
========== : 693 / 5000
========== : 694 / 5000
========== : 695 / 5000
========== : 696 / 5000
========== : 697 / 5000
========== : 698 / 5000
========== : 699 / 5000
========== : 700 / 5000
========== : 701 / 5000
========== : 702 / 5000
========== : 703 / 5000
========== : 704 / 5000
========== : 705 / 5000
========== : 706 / 5000
========== : 707 / 5000
========== : 708 / 5000
========== : 709 / 5000
========== : 710 / 5000
========== : 711 / 5000
========== : 712 / 5000
========== : 713 / 5000
========== : 714 / 5000
========== : 715 / 5000
========== : 716 / 5000
========== : 717 / 5000
========== : 718 / 5000
========== : 719 / 5000
========== : 720 / 5000
========== : 721 / 5000
========== : 722 / 5000
========== : 723 / 5000
========== : 724 / 5000
========== : 725 / 5000
========== : 726 / 5000
========== : 727 / 5000
========== : 728 / 5000
========== : 729 / 5000
========== : 730 / 5000
========== : 731 / 5000
========== : 732 / 5000
========== : 733

========== : 1035 / 5000
========== : 1037 / 5000
========== : 1038 / 5000
========== : 1039 / 5000
========== : 1040 / 5000
========== : 1041 / 5000
========== : 1042 / 5000
========== : 1043 / 5000
========== : 1044 / 5000
========== : 1045 / 5000
========== : 1046 / 5000
========== : 1047 / 5000
========== : 1048 / 5000
========== : 1049 / 5000
========== : 1050 / 5000
========== : 1051 / 5000
========== : 1052 / 5000
========== : 1053 / 5000
========== : 1054 / 5000
========== : 1055 / 5000
========== : 1056 / 5000
========== : 1057 / 5000
========== : 1058 / 5000
========== : 1059 / 5000
========== : 1060 / 5000
========== : 1061 / 5000
========== : 1062 / 5000
========== : 1063 / 5000
========== : 1064 / 5000
========== : 1065 / 5000
========== : 1066 / 5000
========== : 1067 / 5000
========== : 1068 / 5000
========== : 1069 / 5000
========== : 1070 / 5000
========== : 1071 / 5000
========== : 1072 / 5000
========== : 1073 / 5000
========== : 1074 / 5000
========== : 1075 / 5000


KeyboardInterrupt: 

In [29]:
f1 = []
precise = []
recall = []
for item in rouge1:
  f1.append(item['f'])
  precise.append(item['p'])
  recall.append(item['r'])
print(f'f1-mean:{np.array(f1).mean()}, precise-mean:{np.array(precise).mean()}, recall-mean:{np.array(recall).mean()}')

f1-mean:0.39820528118726845, precise-mean:0.449002920699653, recall-mean:0.3737501173385244


In [30]:
f1 = []
precise = []
recall = []
for item in rouge2:
  f1.append(item['f'])
  precise.append(item['p'])
  recall.append(item['r'])
print(f'f1-mean:{np.array(f1).mean()}, precise-mean:{np.array(precise).mean()}, recall-mean:{np.array(recall).mean()}')

f1-mean:0.2348858408233064, precise-mean:0.26439234076537677, recall-mean:0.2210611211559559


In [33]:
f1 = []
precise = []
recall = []
for item in rougeL:
  f1.append(item['f'])
  precise.append(item['p'])
  recall.append(item['r'])
print(f'f1-mean:{np.array(f1).mean()}, precise-mean:{np.array(precise).mean()}, recall-mean:{np.array(recall).mean()}')

f1-mean:0.3324943212339941, precise-mean:0.3748528965517289, recall-mean:0.312369529188253


### 预测

In [ ]:
i = 13
sentence = dataset['test'][i]['content']
summary = dataset['test'][i]['title']
sentence

'6月以来,上证综指先扬后抑,月初小幅震荡上行,突破5100点,随后深幅调整,本周有三个交易日跌幅超百点。在大盘深幅调整之时,权重股未能崛起护盘,特别是银行股延续低迷,昨日尾盘大幅下挫放大了指数跌幅。业内人士认为,目前看,银行股前期滞涨明显,银行混改、混业经营等预期有望重塑银行板块估值体系,银行股估值修复行情依然可期。后市银行股企稳有助股指震荡走稳。融资客加仓银行股本周沪综指连续下挫,跌破5000点、4900点和4800点三个整数关口,连续调整使场内谨慎情绪升温。在大盘调整之际,作为权重股,银行股不仅未能挺身而出,反而震荡下挫,昨日申万银行指数下跌4.43%,而沪综指下跌3.67%。银行股跑输大盘,一定程度上加大了指数调整幅度。6月以来,申万银行指数上涨4.13%,显著跑输其他行业板块。值得注意的是,融资客依然对银行股不离不弃。6月以来,银行板块的融资净买入额为205.92亿元,在28个申万一级行业中位居首位。16只上市银行股中,仅有浦发银行出现7.70亿元的融资净偿还,其余15只银行股均实现融资净买入,其中兴业银行、民生银行、中国银行的融资净买入额居前,均超过20亿元,分别为34.02亿元、27.13亿元和25.25亿元。由此来看,银行股虽然表现低迷,但融资客反而选择积极加仓。成长空间打开业内人士认为,后市银行股估值修复行情可期。首先,国企改革主题将重塑银行股估值体系。银行股低估值显而易见,2015年银行板块市盈率通常维持在7倍左右,属于典型的低估品种,在28个申万一级行业中估值最低。不过,此前银行股的估值修复行情进行得并不顺利,究其原因是此前银行股缺乏估值重塑的契机。目前来看,国企改革主题有望重塑银行股估值体系,打开银行股成长空间。其次,增量资金入场,具备高股息率属性的银行股颇受青睐。2014年上半年沪综指在2000点附近窄幅震荡时,成交额仅在千亿元左右,缺乏增量资金入场,而现有的存量资金很难抬升银行股,银行股此前的估值修复自然难持续。不过,当前资金利率趋于下行,其他大类资金收益率下挫,各类融资产品刚性兑付被打破,原本沉淀其中的资金纷纷流出,配置股票的资金显著增多,增量资金纷纷入场,具备抬起大市值银行股的能力。最后,经济企稳会带动银行股估值修复。作为典型的周期类股票,银行股此前的估值修复往往“无疾而终”与投资者对经济的担忧密不可分。从此前公布的经济数据来看,经济

In [27]:
input = tokenizer(sentence, max_length=max_target_length, truncation=True, return_tensors='pt')
del input['token_type_ids']
output = model.generate(
    **input,
    do_sample=True,
    num_beams=3,
    bad_words_ids=[[101], [100]],
    # length_penalty=100,
    max_length=100,
    repetition_penalty=5.0
)
tokenizer.decode(output[0])

'[CLS] 网 友 称, 新 浪 体 育 显 示 图 片 : " thumb... [SEP] 。 [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]'

In [ ]:
summary

'银行股前期滞涨明显,银行混改、混业经营等预期有望重塑银行板块估值体系,银行股估值修复行情依然可期。'